---
# Apresentação
---
## Resumo
> Suponha uma empresa de turismo em uma grande cidade e que há um turista que deseja conhecer essa cidade. Entretanto, ele elencou apenas 1 ponto que deseja visitar. A proposta do projeto é desenvolver um algoritmo capaz de gerar uma rota a partir da sede da empresa e que passe por uma quantidade mínima de pontos turísticos, de modo que o ponto final seja o ponto escolhido pelo cliente. Assim, ele poderá conhecer mais lugares da cidade, além daquele previamente escolhido.

## Objetivo
> O presente arquivo visa utilizar algoritmos de busca cega e heurística para encontrar um caminho dado um ponto turístico de destino.

## Notas
> Este Código foi desenvolvido em Python no Google Colaboratory e seu funcionamento está a ele relacionado.

> **Para um melhor acompanhamento**, [visualize através do Colab](https://colab.research.google.com/drive/1jnO68wCK8EXQG_mDDanYRnfgbbPeYqYq?usp=sharing) utilizando os índices.

---
# Bibliotecas, funções e constantes
---

In [14]:
# @title Importando as bibliotecas necessárias

# Matemática
import math
import numpy as np
import pandas as pd

# Manipulação
from time import sleep
from pprint import pprint
import folium
import heapq

In [15]:
# @title Definindo as Constantes

ORIGEM =  'Sede da empresa (Ponto inicial)'

DESTINO = 'Cristo Redentor'

QUANTIDADE_MINIMA_DE_PONTOS = 4

In [16]:
# @title Código para nome

def codigo_para_nome(codigo: str, pontos_turisticos: pd.DataFrame) -> str:
    return pontos_turisticos.loc[pontos_turisticos['Place ID'] == codigo, 'Name'].values[0]

In [17]:
# @title Nome para código

def nome_para_codigo(nome: str, pontos_turisticos: pd.DataFrame) -> str:
    return pontos_turisticos.loc[pontos_turisticos['Name'] == nome, 'Place ID'].values[0]

In [18]:
# @title Função para obter Função de Custo - FC

def obtem_fc(origem: str, destino: str) -> float:
  d_real_n = df_distancias_reais.at[origem, destino]
  t_real_n = df_tempo_carro_horas.at[origem, destino]
  return ((5.93 / 2.215) * d_real_n * t_real_n)

In [19]:
# @title Função para obter Função de Avaliação - FA

def obtem_fa(origem: str, destino: str) -> float:
  d_final_n = df_distancias_estimadas.at[origem, destino]
  return (5.93 / (2.215 * 90)) * d_final_n**2

In [20]:
# @title Função para criar um mapa com marcadores para todos os pontos turístico

def cria_mapa_com_pontos(todos_os_pontos: pd.DataFrame, com_raio: bool = True):
  mapa = folium.Map(location=[df_pontos_turisticos['Latitude'].mean(), df_pontos_turisticos['Longitude'].mean()], zoom_start=12)

  for index, row in df_pontos_turisticos.iterrows():
    if row['Name'] == 'Sede da empresa (Ponto inicial)':
      folium.Marker(
          [row['Latitude'], row['Longitude']],
          popup=row['Name'],
          icon=folium.Icon(color='green',icon_color='#F7F7F7', prefix='fa', icon='building')
      ).add_to(mapa)

      if com_raio:
        # Adiciona um círculo de 10 km de raio ao redor da sede
        folium.Circle(
            radius=10000,  # Raio em metros
            location=[row['Latitude'], row['Longitude']],
            popup='Área de 10 km',
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.1
        ).add_to(mapa)
    else:
      folium.Marker(
          [row['Latitude'], row['Longitude']],
          popup=row['Name'],
          icon=folium.Icon(color='red', prefix='fa', icon='camera')
      ).add_to(mapa)

  return mapa

In [21]:
# @title Função para adicionar conexoes em um mapa com marcadores

def adiciona_conexoes_em_um_mapa_com_pontos(mapa, grafo: dict[str, list], densidade_linha=1, opacidade_linha=0.25, caminho_especial=None, mostrar_conexoes: bool = True) -> None:
    caminho_especial_set = set(zip(caminho_especial, caminho_especial[1:])) if caminho_especial else set()
    for place_id, grafo_ids in grafo.items():
        ponto_origem = df_pontos_turisticos.loc[df_pontos_turisticos['Place ID'] == place_id, ['Latitude', 'Longitude']].values[0]
        for conexao_id in grafo_ids:
            ponto_destino = df_pontos_turisticos.loc[df_pontos_turisticos['Place ID'] == conexao_id, ['Latitude', 'Longitude']].values[0]
            color = "blue"
            weight = densidade_linha
            opacity = opacidade_linha
            if (place_id, conexao_id) in caminho_especial_set or (conexao_id, place_id) in caminho_especial_set:
                color = "red"
                weight = 5  # Destaca o caminho especial
                opacity = 0.8
            if mostrar_conexoes or color == "red":
              folium.PolyLine(
                  locations=[ponto_origem, ponto_destino],
                  color=color,
                  weight=weight,
                  opacity=opacity
              ).add_to(mapa)

In [22]:
# @title Função para criar mapa com pontos e caminho gerado

def constroi_mapa_com_pontos_e_caminho(todos_os_pontos: pd.DataFrame, grafo: dict[str, list], caminho_final: list[str], mostrar_conexoes: bool = True):
    mapa = cria_mapa_com_pontos(todos_os_pontos, com_raio=False)
    adiciona_conexoes_em_um_mapa_com_pontos(mapa, grafo, caminho_especial=caminho_final, mostrar_conexoes = mostrar_conexoes)

    return mapa

---
# Preliminares
---

In [23]:
# @title Lendo datasets de dados

df_pontos_turisticos = pd.read_excel('./pontos_turisticos-rio.xlsx')
df_distancias_reais = pd.read_excel('./matriz_distancia_carro_km-rio.xlsx', index_col='Place ID')
df_distancias_estimadas = pd.read_excel('./matriz_distancia_euclidiana_km-rio.xlsx', index_col='Place ID')
df_tempo_carro_horas = pd.read_excel('./matriz_tempo_carro_horas-rio.xlsx', index_col='Place ID')

In [24]:
# @title Criando um grafo (Hiperconectado)

grafo = {place_id: list(df_distancias_reais.columns) for place_id in df_distancias_reais.index}

In [25]:
# @title Renomeando Origem e destino para códigos

ORIGEM = nome_para_codigo(ORIGEM, df_pontos_turisticos)
DESTINO = nome_para_codigo(DESTINO, df_pontos_turisticos)

---
# Busca Cega - Em Profundidade
---

In [26]:
# @title Implementação da Busca Cega - DFS

def dfs(grafo: dict[str, list], origem: str, destino: str) -> tuple[float, int, list]:
    caminho = []
    pilha = [(origem, [origem], 0)] # (ponto, caminho, custo total)
    nos_visitados = 0

    while pilha:
        atual, caminho, custo_total_km = pilha.pop()
        nos_visitados += 1

        if atual == destino:
            return custo_total_km, nos_visitados, caminho

        for vizinho in grafo[atual]:
            custo_vizinho = df_distancias_reais.at[atual, vizinho]
            if vizinho not in caminho:
                novo_custo = custo_total_km + custo_vizinho
                pilha.append((vizinho, caminho + [vizinho], novo_custo))

    return 0, 0, []

custo_total_km, numero_de_nos_visitados, caminho_final = dfs(grafo, ORIGEM, DESTINO)

In [27]:
tempo_total_horas = 0
for i in range(len(caminho_final) - 1):
  ponto_a = caminho_final[i]
  ponto_b = caminho_final[i + 1]
  tempo_total_horas += df_tempo_carro_horas.at[ponto_a, ponto_b]

d_real_n = custo_total_km
t_real_n = tempo_total_horas
custo_total_heuristica = (5.93 / 2.215) * d_real_n * t_real_n

In [28]:
print(custo_total_km,custo_total_heuristica, numero_de_nos_visitados)

356.17200000000014 11346.114106666671 38


In [29]:
# @title Resultado final

mapa = constroi_mapa_com_pontos_e_caminho(
    todos_os_pontos=df_pontos_turisticos,
    grafo=grafo,
    caminho_final=caminho_final
  )

mapa.save("caminho_gerado_profundidade.html")
mapa

---
# Busca Informada - Heurística
---

In [30]:
# @title Implementação da Busca Informada - A*

def a_estrela(grafo: dict[str, list], origem: str, destino: str, min_pontos: int) -> tuple[float, int, int, list[str]]:
    fila = []
    heapq.heappush(fila, (0, origem, [origem], 0))  # (custo estimado f, ponto, caminho, custo real g)
    nos_visitados = nos_abertos = 0

    while fila:
        custo_total_heuristica, atual, caminho, g = heapq.heappop(fila)
        nos_visitados += 1

        if atual == destino and len(caminho) == min_pontos + 2:
            return custo_total_heuristica, nos_visitados, nos_abertos, caminho

        if len(caminho) < min_pontos + 2:
          for vizinho in grafo[atual]:
            if vizinho not in caminho:
              novo_g = g + obtem_fc(atual, vizinho)
              h = obtem_fa(vizinho,destino)
              f = novo_g + h
              nos_abertos += 1
              heapq.heappush(fila, (f, vizinho, caminho + [vizinho], novo_g))

    return 0, 0, 0, []

custo_total_heuristica, numero_de_nos_visitados, numero_de_nos_abertos, caminho_final = a_estrela(grafo, ORIGEM, DESTINO, QUANTIDADE_MINIMA_DE_PONTOS)

In [31]:
custo_total_km = 0
for i in range(len(caminho_final) - 1):
  ponto_a = caminho_final[i]
  ponto_b = caminho_final[i + 1]
  custo_total_km += df_distancias_reais.at[ponto_a, ponto_b]

In [32]:
print(custo_total_heuristica, custo_total_km, numero_de_nos_visitados, numero_de_nos_abertos)

14.914946513669427 21.914 1290687 4598230


In [33]:
# @title Resultado final

mapa = constroi_mapa_com_pontos_e_caminho(
    todos_os_pontos=df_pontos_turisticos,
    grafo=grafo,
    caminho_final=caminho_final
  )

mapa.save("caminho_gerado_a_estrela_5_pontos.html")
mapa